# Import library

In [1]:
import sys
sys.path.append('src')
import enex_analysis as enex
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import dartwork_mpl as dm
import warnings
import matplotlib.colors as mcolors
import matplotlib.ticker as ticker
warnings.filterwarnings("ignore")

Load colors...
Load colormaps...


# System setting

In [4]:
ASHPB = enex.HeatPumpBoiler()
ASHPB.T0 = 10
# Efficiency [-]
ASHPB.eta_fan = 0.6
        
# Pressure [Pa]
ASHPB.dP = 200 

# Temperature [K]
ASHPB.T0          = 0
ASHPB.T_a_ext_out = ASHPB.T0 - 5
ASHPB.T_r_ext     = ASHPB.T0 - 10

ASHPB.T_w_tank    = 60
ASHPB.T_r_tank    = ASHPB.T_w_tank + 5

ASHPB.T_w_serv    = 45
ASHPB.T_w_sup     = 10

# Tank water use [L/min]
ASHPB.dV_w_serv  = 1.2

# Tank size [m]
ASHPB.r0 = 0.2
ASHPB.H = 0.8

# Tank layer thickness [m]
ASHPB.x_shell = 0.01 
ASHPB.x_ins   = 0.10 

# Tank thermal conductivity [W/mK]
ASHPB.k_shell = 25   
ASHPB.k_ins   = 0.03 

# Overall heat transfer coefficient [W/m²K]
ASHPB.h_o = 15 

# Maximum heat transfer from refrigerant to tank water [W]
ASHPB.Q_r_max = 4000 

ASHPB.system_update()

In [5]:
T0_range = np.arange(-10, 30, 0.1)
T_w_serv_range = np.arange(30, 50, 0.1)

# Data

In [6]:
X_eff_dict = {
    'external unit': np.zeros((len(T0_range), len(T_w_serv_range))),
    'refrigerant': np.zeros((len(T0_range), len(T_w_serv_range))),
    'hot water tank': np.zeros((len(T0_range), len(T_w_serv_range))),
    'mixing valve': np.zeros((len(T0_range), len(T_w_serv_range))),
    'total': np.zeros((len(T0_range), len(T_w_serv_range)))
             }
PLR = np.zeros((len(T0_range), len(T_w_serv_range)))
COP = np.zeros((len(T0_range), len(T_w_serv_range)))

# np.zeros((len(T0_range), len(T_w_serv_range)))
for i, T0 in enumerate(T0_range): # row
    for j, T_w_serv in enumerate(T_w_serv_range): # col
        ASHPB = enex.HeatPumpBoiler()
        ASHPB.T0 = T0
        ASHPB.T_w_serv = T_w_serv
        ASHPB.T_a_ext_out = T0 - 5
        ASHPB.T_r_ext     = T0 - 10
        ASHPB.T_r_tank    = 65
        ASHPB.system_update()
        X_eff_dict['total'][i, j] = ASHPB.X_eff*100
        # External unit exergy efficiency: X_a_ext_out / (E_fan + X_r_ext + X_a_ext_in)
        X_eff_dict['external unit'][i, j] = (
            ASHPB.X_a_ext_out / (ASHPB.E_fan + ASHPB.X_r_ext + ASHPB.X_a_ext_in) * 100
        )
        # Refrigerant loop exergy efficiency: (X_r_tank + X_r_ext) / E_cmp
        X_eff_dict['refrigerant'][i, j] = (
            (ASHPB.X_r_tank + ASHPB.X_r_ext) / ASHPB.E_cmp * 100
        )
        # Hot water tank exergy efficiency: (X_w_tank + X_l_tank) / (X_r_tank + X_w_sup_tank)
        X_eff_dict['hot water tank'][i, j] = (
            (ASHPB.X_w_tank) / (ASHPB.X_r_tank + ASHPB.X_w_sup_tank) * 100
        )
        # Mixing valve exergy efficiency: X_w_serv / (X_w_tank + X_w_sup_mix)
        X_eff_dict['mixing valve'][i, j] = (
            ASHPB.X_w_serv / (ASHPB.X_w_tank + ASHPB.X_w_sup_mix) * 100
        )
        COP[i, j] = ASHPB.COP
        PLR[i, j] = ASHPB.Q_r_tank/ ASHPB.Q_r_max*100

# Figure setting

In [7]:
plt.rcParams['font.size'] = 12

fs_dict = {
    'label': dm.fs(0),
    'tick': dm.fs(-1),
    'legend': dm.fs(-1.5),
    'annotation': dm.fs(-1),
            }

pad = {
    'label': 6,
    'tick': 3,
}

layout ={
    'bbox': (0.1, 0.1, 0.8, 0.8),
    'margins': (0, 0, 0, 0),
}

LW = np.arange(0.25, 3.0, 0.25)

## Sub system exergy efficiency heatmap

In [8]:
# Plot ====================================================================================================
nrows = 1
ncols = 4
nfigs = nrows * ncols

fig, ax = plt.subplots(
    nrows, ncols, 
    sharex=True, sharey=True, 
    figsize=(dm.SW*4, dm.SW), 
    facecolor='none', edgecolor='k',
    squeeze=False,
    dpi=300,
)
# tick settings 
xmin = [np.min(T_w_serv_range)] * nfigs
xmax = [T_w_serv_range[-1]] * nfigs
xint = [5] * nfigs
xmar = [0] * nfigs

ymin = [np.min(T0_range)] * nfigs
ymax = [T0_range[-1]] * nfigs
yint = [10] * nfigs
ymar = [0] * nfigs
cmax = 100

eff_keys = ['external unit', 'refrigerant', 'hot water tank', 'mixing valve']
subplot_titles = ['(a) External Unit', '(b) Refrigerant', '(c) Hot Water Tank', '(d) Mixing Valve']

for cidx in range(ncols): 
    idx = cidx
    norm = mcolors.Normalize(vmin=0, vmax=cmax)
    eff_mat = X_eff_dict[eff_keys[cidx]]

    im = ax[0, cidx].imshow(
        eff_mat,
        cmap='dm.jet',
        norm=norm,
        extent=[T_w_serv_range[0], T_w_serv_range[-1], T0_range[0], T0_range[-1]],
        aspect='auto',
        interpolation='none',
        origin='lower',
    )

    ax[0, cidx].set_xlim(xmin[idx] - xmar[idx], xmax[idx] + xmar[idx])
    ax[0, cidx].set_ylim(ymin[idx] - ymar[idx], ymax[idx] + ymar[idx])

    ax[0, cidx].set_xlabel('T_w_serv', fontsize=fs_dict['label'])
    ax[0, 0].set_ylabel('T0', fontsize=fs_dict['label'])

    ax[0, cidx].annotate(subplot_titles[cidx], xy=(.01, 1.01), xycoords='axes fraction',
        horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 

    ax[0, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='major', length=2.5, width=0.5)
    ax[0, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='minor', length=1.25, width=0.25)

    ax[0, cidx].set_xticks(np.arange(xmin[idx], xmax[idx] + xint[idx], xint[idx]))
    ax[0, cidx].set_yticks(np.arange(ymin[idx], ymax[idx] + yint[idx], yint[idx]))

    ax[0, cidx].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
    ax[0, cidx].yaxis.set_minor_locator(ticker.AutoMinorLocator(3))
    ax[0, cidx].grid(False)

    for k in ['top', 'bottom', 'left', 'right']:
        ax[0, cidx].spines[k].set_visible(True)
        ax[0, cidx].spines[k].set_linewidth(0.5)
        ax[0, cidx].spines[k].set_color('k')

dm.simple_layout(fig, bbox=[0.01, 0.95, 0.1, 1.0], margins=[0.0, 0.1, 0.05, 0.05])

# colorbar (공통 colorbar 하나만 추가)
bbox1 = ax[0, 0].get_position()
bbox3 = ax[0,-1].get_position()

# colorbar
cbar_width =  0.016 #vertical  
cbar_height = 0.010 #horizontal  
cbar_dist_v = 0.08; # vertical colorbar distance from bbox edge
cbar_dist_h = 0.02; # horizontal colorbar distance from bbox edge

cb_ax1 = fig.add_axes([bbox3.x1 + cbar_dist_h, bbox1.y0, cbar_width, bbox3.y1 - bbox3.y0]) #[x_origin, y_origin, width, height]
cbar1 = fig.colorbar(im, cax=cb_ax1, ax=ax.ravel().tolist(), orientation='vertical')
minor_locator = ticker.MultipleLocator(cmax / 5)
cbar1.ax.yaxis.set_minor_locator(minor_locator)
cbar1.ax.tick_params(which="major", direction='in', labelsize=fs_dict['tick'], length=2, width=0.25)
cbar1.ax.minorticks_off()
cbar1.locator = ticker.MultipleLocator(cmax / 5)
cbar1.ax.set_ylabel('Exergy Efficiency', fontsize=fs_dict['label'])
cbar1.update_ticks()
cbar1.outline.set_linewidth(0.5)

plt.close()
dm.save_and_show(fig)


# COP

In [12]:
# Plot ====================================================================================================
nrows = 1
ncols = 1
nfigs = nrows * ncols

fig, ax = plt.subplots(
    nrows, ncols, 
    sharex=True, sharey=True, 
    figsize=(dm.SW, dm.SW), 
    facecolor='none', edgecolor='k',
    squeeze=False,
    dpi=300,
)

# x축을 T_w_serv, y축을 T0로 변경
xmin = [np.min(T_w_serv_range)] * nfigs
xmax = [T_w_serv_range[-1]] * nfigs
xint = [5] * nfigs
xmar = [0] * nfigs

ymin = [np.min(T0_range)] * nfigs
ymax = [T0_range[-1]] * nfigs
yint = [10] * nfigs
ymar = [0] * nfigs
cmax = 8

for cidx in range(ncols): 
    idx = cidx
    norm = mcolors.Normalize(vmin=2, vmax=cmax)
    eff_mat = COP

    im = ax[0, cidx].imshow(
        eff_mat,  # 전치하여 x/y축 맞춤
        cmap='dm.BuPu',
        norm=norm,
        extent=[T_w_serv_range[0], T_w_serv_range[-1], T0_range[0], T0_range[-1]],
        aspect='auto',
        interpolation='none',
        origin='lower',
    )

    ax[0, cidx].set_xlim(xmin[idx] - xmar[idx], xmax[idx] + xmar[idx])
    ax[0, cidx].set_ylim(ymin[idx] - ymar[idx], ymax[idx] + ymar[idx])

    ax[0, cidx].set_xlabel('T_w_serv', fontsize=fs_dict['label'])
    if cidx == 0:
        ax[0, cidx].set_ylabel('T0', fontsize=fs_dict['label'])

    ax[0, cidx].annotate('(a) COP', xy=(.01, 1.01), xycoords='axes fraction',
        horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 

    ax[0, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='major', length=2.5, width=0.5)
    ax[0, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='minor', length=1.25, width=0.25)

    ax[0, cidx].set_xticks(np.arange(xmin[idx], xmax[idx] + xint[idx], xint[idx]))
    ax[0, cidx].set_yticks(np.arange(ymin[idx], ymax[idx] + yint[idx], yint[idx]))

    ax[0, cidx].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
    ax[0, cidx].yaxis.set_minor_locator(ticker.AutoMinorLocator(3))
    ax[0, cidx].grid(False)

    for k in ['top', 'bottom', 'left', 'right']:
        ax[0, cidx].spines[k].set_visible(True)
        ax[0, cidx].spines[k].set_linewidth(0.5)
        ax[0, cidx].spines[k].set_color('k')

dm.simple_layout(fig, bbox=[0.01, 0.9, 0.01, 1.0], margins=[0.0, 0.1, 0.05, 0.05])

# colorbar (공통 colorbar 하나만 추가)
bbox1 = ax[0, 0].get_position()
bbox3 = ax[0,-1].get_position()

# colorbar
cbar_width =  0.04 #vertical  
cbar_height = 0.010 #horizontal  
cbar_dist_v = 0.08; # vertical colorbar distance from bbox edge
cbar_dist_h = 0.03; # horizontal colorbar distance from bbox edge

cb_ax1 = fig.add_axes([bbox3.x1 + cbar_dist_h, bbox1.y0, cbar_width, bbox3.y1 - bbox3.y0]) 
cbar1 = fig.colorbar(im, cax=cb_ax1, ax=ax.ravel().tolist(), orientation='vertical')
minor_locator = ticker.MultipleLocator(cmax / 4)
cbar1.ax.yaxis.set_minor_locator(minor_locator)
cbar1.ax.tick_params(which="major", direction='in', labelsize=fs_dict['tick'], length=2, width=0.25)
cbar1.ax.minorticks_off()
cbar1.locator = ticker.MultipleLocator(cmax / 4)
cbar1.ax.set_ylabel('COP', fontsize=fs_dict['label'])
cbar1.update_ticks()
cbar1.outline.set_linewidth(0.5)

# Add white contour lines
cs = ax[0, 0].contour(
    T_w_serv_range, T0_range, COP, 
    levels=np.arange(2, cmax, 1), 
    colors='white', linewidths=1, linestyles='solid'
)
ax[0, 0].clabel(cs, fmt='%1.1f', colors='white', fontsize=fs_dict['tick'])

plt.close()
dm.save_and_show(fig)


# PLR

In [ ]:
# Plot ====================================================================================================
nrows = 1
ncols = 1
nfigs = nrows * ncols

fig, ax = plt.subplots(
    nrows, ncols, 
    sharex=True, sharey=True, 
    figsize=(dm.SW, dm.SW), 
    facecolor='none', edgecolor='k',
    squeeze=False,
    dpi=300,
)

# x축을 T_w_serv, y축을 T0로 변경
xmin = [np.min(T_w_serv_range)] * nfigs
xmax = [T_w_serv_range[-1]] * nfigs
xint = [5] * nfigs
xmar = [0] * nfigs

ymin = [np.min(T0_range)] * nfigs
ymax = [T0_range[-1]] * nfigs
yint = [10] * nfigs
ymar = [0] * nfigs
cmax = 100

for cidx in range(ncols): 
    idx = cidx
    norm = mcolors.Normalize(vmin=40, vmax=cmax)
    eff_mat = PLR

    im = ax[0, cidx].imshow(
        eff_mat,  # 전치하여 x/y축 맞춤
        cmap='dm.Blues9',
        norm=norm,
        extent=[T_w_serv_range[0], T_w_serv_range[-1], T0_range[0], T0_range[-1]],
        aspect='auto',
        interpolation='none',
        origin='lower',
    )

    ax[0, cidx].set_xlim(xmin[idx] - xmar[idx], xmax[idx] + xmar[idx])
    ax[0, cidx].set_ylim(ymin[idx] - ymar[idx], ymax[idx] + ymar[idx])

    ax[0, cidx].set_xlabel('T_w_serv', fontsize=fs_dict['label'])
    if cidx == 0:
        ax[0, cidx].set_ylabel('T0', fontsize=fs_dict['label'])

    ax[0, cidx].annotate('(a) PLR', xy=(.01, 1.01), xycoords='axes fraction',
        horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 

    ax[0, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='major', length=2.5, width=0.5)
    ax[0, cidx].tick_params(direction='in', labelsize=fs_dict['label'], which='minor', length=1.25, width=0.25)

    ax[0, cidx].set_xticks(np.arange(xmin[idx], xmax[idx] + xint[idx], xint[idx]))
    ax[0, cidx].set_yticks(np.arange(ymin[idx], ymax[idx] + yint[idx], yint[idx]))

    ax[0, cidx].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
    ax[0, cidx].yaxis.set_minor_locator(ticker.AutoMinorLocator(3))
    ax[0, cidx].grid(False)


dm.simple_layout(fig, bbox=[0.01, 0.9, 0.0, 1.0], margins=[0.0, 0.1, 0.05, 0.05])

# colorbar (공통 colorbar 하나만 추가)
bbox1 = ax[0, 0].get_position()
bbox3 = ax[0,-1].get_position()

# colorbar
cbar_width =  0.04 #vertical  
cbar_height = 0.010 #horizontal  
cbar_dist_v = 0.08; # vertical colorbar distance from bbox edge
cbar_dist_h = 0.03; # horizontal colorbar distance from bbox edge

cb_ax1 = fig.add_axes([bbox3.x1 + cbar_dist_h, bbox1.y0, cbar_width, bbox3.y1 - bbox3.y0]) 

# colorbar (공통 colorbar 하나만 추가)
cbar1 = fig.colorbar(im, cax=cb_ax1, ax=ax.ravel().tolist(), orientation='vertical')
minor_locator = ticker.MultipleLocator(cmax / 5)
cbar1.ax.yaxis.set_minor_locator(minor_locator)
cbar1.ax.tick_params(which="major", direction='in', labelsize=fs_dict['tick'], length=2, width=0.25)
cbar1.ax.minorticks_off()
cbar1.locator = ticker.MultipleLocator(cmax / 5)
cbar1.ax.set_ylabel('PLR', fontsize=fs_dict['label'])
cbar1.update_ticks()
cbar1.outline.set_linewidth(0.5)

# Add white contour lines
cs = ax[0, 0].contour(
    T_w_serv_range, T0_range, PLR, 
    levels=np.arange(0, cmax, 10), 
    colors='white', linewidths=1, linestyles='solid'
)
ax[0, 0].clabel(cs, fmt='%1.1f', colors='white', fontsize=fs_dict['tick'])

plt.close()
dm.save_and_show(fig)


## PLR & COP

In [35]:
# Plot ====================================================================================================
nrows = 1
ncols = 2
nfigs = nrows * ncols

fig, ax = plt.subplots(
    nrows, ncols, 
    sharex=True, sharey=True, 
    figsize=(dm.SW * 2.2, dm.SW),  # 가로 두배
    facecolor='none', edgecolor='k',
    squeeze=False,
    dpi=300,
)

# x축을 T_w_serv, y축을 T0로 변경
xmin = [np.min(T_w_serv_range)] * nfigs
xmax = [T_w_serv_range[-1]] * nfigs
xint = [5] * nfigs
xmar = [0] * nfigs

ymin = [np.min(T0_range)] * nfigs
ymax = [T0_range[-1]] * nfigs
yint = [10] * nfigs
ymar = [0] * nfigs

data_list = [PLR, COP]
cmap_list = ['dm.Blues9', 'dm.BuPu']
norm_list = [mcolors.Normalize(vmin=40, vmax=100), mcolors.Normalize(vmin=2, vmax=8)]
cmax_list = [100, 8]
levels_list = [np.arange(0, 100, 10), np.arange(2, 8, 1)]
labels = ['PLR', 'COP']
annotations = ['(a) PLR', '(b) COP']

im_list = []
cs_list = []

for i in range(nrows):
    for j in range(ncols):
        idx = i * ncols + j
        im = ax[i, j].imshow(
            data_list[idx],
            cmap=cmap_list[idx],
            norm=norm_list[idx],
            extent=[T_w_serv_range[0], T_w_serv_range[-1], T0_range[0], T0_range[-1]],
            aspect='auto',
            interpolation='none',
            origin='lower',
        )
        im_list.append(im)
        ax[i, j].set_xlim(xmin[idx] - xmar[idx], xmax[idx] + xmar[idx])
        ax[i, j].set_ylim(ymin[idx] - ymar[idx], ymax[idx] + ymar[idx])
        ax[i, j].set_xlabel('T_w_serv', fontsize=fs_dict['label'])
        if j == 0:
            ax[i, j].set_ylabel('T0', fontsize=fs_dict['label'])
        ax[i, j].annotate(annotations[idx], xy=(.01, 1.01), xycoords='axes fraction',
            horizontalalignment='left', verticalalignment='bottom', fontsize=fs_dict['annotation']) 
        ax[i, j].tick_params(direction='in', labelsize=fs_dict['label'], which='major', length=2.5, width=0.5)
        ax[i, j].tick_params(direction='in', labelsize=fs_dict['label'], which='minor', length=1.25, width=0.25)
        ax[i, j].set_xticks(np.arange(xmin[idx], xmax[idx] + xint[idx], xint[idx]))
        ax[i, j].set_yticks(np.arange(ymin[idx], ymax[idx] + yint[idx], yint[idx]))
        ax[i, j].xaxis.set_minor_locator(ticker.AutoMinorLocator(5))
        ax[i, j].yaxis.set_minor_locator(ticker.AutoMinorLocator(3))
        ax[i, j].grid(False)

        for k in ['top', 'bottom', 'left', 'right']:
            ax[0, j].spines[k].set_visible(True)
            ax[0, j].spines[k].set_linewidth(0.5)
            ax[0, j].spines[k].set_color('k')

        cs = ax[i, j].contour(
            T_w_serv_range, T0_range, data_list[idx], 
            levels=levels_list[idx], 
            colors='white', linewidths=1, linestyles='solid'
        )
        cs_list.append(cs)
        ax[i, j].clabel(cs, fmt='%1.1f', colors='white', fontsize=fs_dict['tick'])

# Increase horizontal spacing between subplots
fig.subplots_adjust(wspace=0.3)

dm.simple_layout(fig, bbox=[0.01, 0.95, 0.01, 1.0], margins=[0.0, 0.1, 0.05, 0.05])
# colorbar (PLR)
cbar_width = 0.02  # 너비를 반으로 줄임
cbar_dist_h = 0.02  # ax 우측에 가깝게

bbox1 = ax[0, 0].get_position()
cb_ax1 = fig.add_axes([bbox1.x1 + cbar_dist_h, bbox1.y0, cbar_width, bbox1.y1 - bbox1.y0])
cbar1 = fig.colorbar(im_list[0], cax=cb_ax1, ax=ax[0, 0], orientation='vertical')
minor_locator1 = ticker.MultipleLocator(cmax_list[0] / 5)
cbar1.ax.yaxis.set_minor_locator(minor_locator1)
cbar1.ax.tick_params(which="major", direction='in', labelsize=fs_dict['tick'], length=2, width=0.25)
cbar1.ax.minorticks_off()
cbar1.locator = ticker.MultipleLocator(cmax_list[0] / 5)
cbar1.ax.set_ylabel(labels[0], fontsize=fs_dict['label'])
cbar1.update_ticks()
cbar1.outline.set_linewidth(0.5)

# colorbar (COP)
bbox2 = ax[0, 1].get_position()
cb_ax2 = fig.add_axes([bbox2.x1 + cbar_dist_h, bbox2.y0, cbar_width, bbox2.y1 - bbox2.y0])
cbar2 = fig.colorbar(im_list[1], cax=cb_ax2, ax=ax[0, 1], orientation='vertical')
minor_locator2 = ticker.MultipleLocator(cmax_list[1] / 4)
cbar2.ax.yaxis.set_minor_locator(minor_locator2)
cbar2.ax.tick_params(which="major", direction='in', labelsize=fs_dict['tick'], length=2, width=0.25)
cbar2.ax.minorticks_off()
cbar2.locator = ticker.MultipleLocator(cmax_list[1] / 4)
cbar2.ax.set_ylabel(labels[1], fontsize=fs_dict['label'])
cbar2.update_ticks()
cbar2.outline.set_linewidth(0.5)

plt.close()
dm.save_and_show(fig)
